In [1]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.layers import Dense, GlobalAveragePooling2D, merge
from keras import backend as K
import numpy as np
from keras.callbacks import Callback
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

import preprocess as ppx

Using TensorFlow backend.


In [3]:
# a képek betöltése és előfeldolgozása
image_file_names, data = ppx.csv_load()
image_name_dict, images = ppx.load_images(image_file_names, "/media/bence/121A62041A61E4E7/learn/train_sample")
train_images, train_data = ppx.data_preprocess(images, data, image_name_dict)

In [4]:
train_images.shape

(8000, 256, 256, 3)

In [5]:
# adatok megfelelő formátumra hozása a keras számára
labels = np.array([ls[1] for ls in train_data])

encoder = preprocessing.LabelEncoder()
encoder.fit(labels)

encoded_l = encoder.transform(labels)
print(encoded_l)

labels_onehot = to_categorical(encoded_l)
print(labels_onehot)

[ 102  724  373 ...,  706 1256  813]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [6]:
class TrainingHistory(Callback):
    # Tanulási folyamat elején létrehozunk egy-egy üres listát a kinyerni kívánt metrikák tárolása céljából.
    def on_train_begin(self, logs={}):
        # Hiba mértéke a tanító adatokon.
        self.losses = []
        # Hiba mértéke a validációs adatokon.
        self.valid_losses = []
        # A modell jóságát, pontosságát mérő mutatószám a tanító adatokon. 
        self.accs = []
        # A modell jóságát, pontosságát mérő mutatószám a validációs adatokon. 
        self.valid_accs = []
        # A tanítási fázisok sorszámozása.
        self.epoch = 0
    
    # Minden egyes tanítási fázis végén mentsük el, hogy hogyan teljesít aktuálisan a háló. 
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 1 == 0:
            self.losses.append(logs.get('loss'))
            self.valid_losses.append(logs.get('val_loss'))
            self.accs.append(logs.get('acc'))
            self.valid_accs.append(logs.get('val_acc'))
            self.epoch += 1
            
history = TrainingHistory()

In [7]:
##########################################################
# error esetére, elvileg nem okoz gondot 'jó' esetben sem
import tensorflow as tf
tf.python.control_flow_ops = tf
##########################################################

# előtanított modell betöltése, a fully-connected rétegek nélkül
base_model = InceptionV3(weights='imagenet', include_top=False)
# az utolsó konvolúciós réteg utána egy global average pooling réteget teszünk, ez rögtön "lapítja" (flatten) a 2D konvolúciót


In [8]:
# kinyerjük a stílusjegyeket a cnn köztes rétegegeiből (és max pool cnn kimeneti rétegére)
style1 = base_model.layers[54 ].output
style1 = GlobalAveragePooling2D()(style1)
style1 = Dense(96, activation='relu')(style1)
style2 = base_model.layers[117].output
style2 = GlobalAveragePooling2D()(style2)
style2 = Dense(160, activation='relu')(style2)
style3 = base_model.layers[184].output
style3 = GlobalAveragePooling2D()(style3)
style3 = Dense(320, activation='relu')(style3)

style_final = base_model.output
style_final = GlobalAveragePooling2D()(style_final)

# egymás mellé tesszük a különböző szintű feature-öket
ff = merge([style1, style2, style3, style_final], mode='concat')

# ezután hozzáadunk két előrecsatolt réteget ReLU aktivációs függvénnyel
ff = Dense(1024, activation='relu')(ff)
ff = Dense(1024, activation='relu')(ff)
ff = Dense(1024, activation='relu')(ff)

# és végül egy kimenete lesz a hálónak - a "binary_crossentropy" költségfüggvénynek erre van szüksége
predictions = Dense(labels_onehot.shape[1], activation='softmax')(ff)
# a model létrehozása
model = Model(input=base_model.input, output=predictions)

tf
tf
tf
tf


In [9]:
# két lépésben fogjuk tanítani a hálót
# az első lépésben csak az előrecsatolt rétegeket tanítjuk, a konvolúciós rétegeket befagyasztjuk
for layer in base_model.layers:
    layer.trainable = False
# lefordítjuk a modelt (fontos, hogy ezt a rétegek befagyasztása után csináljuk"
# mivel két osztályunk van, ezért bináris keresztentrópia költségfüggvényt használunk
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(train_images, labels_onehot, batch_size=8, nb_epoch=50, validation_split=0.2, callbacks=[history])

Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 200s - loss: 6.8438 - acc: 0.0127 - val_loss: 7.3308 - val_acc: 0.0131
Epoch 2/50
6400/6400 [==============================] - 196s - loss: 6.3816 - acc: 0.0236 - val_loss: 7.0970 - val_acc: 0.0194
Epoch 3/50
6400/6400 [==============================] - 197s - loss: 6.1089 - acc: 0.0303 - val_loss: 7.1723 - val_acc: 0.0238
Epoch 4/50
6400/6400 [==============================] - 196s - loss: 5.8878 - acc: 0.0416 - val_loss: 7.2343 - val_acc: 0.0300
Epoch 5/50
6400/6400 [==============================] - 196s - loss: 5.7486 - acc: 0.0519 - val_loss: 7.1477 - val_acc: 0.0325
Epoch 6/50
6400/6400 [==============================] - 196s - loss: 5.5625 - acc: 0.0633 - val_loss: 7.4856 - val_acc: 0.0375
Epoch 7/50
6400/6400 [==============================] - 197s - loss: 5.4162 - acc: 0.0766 - val_loss: 7.4649 - val_acc: 0.0456
Epoch 8/50
6400/6400 [==============================] - 197s - 

In [11]:
# ehhez először nézzük meg a háló felépítését
print("Az Inception V3 konvolúciós rétegei:")
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

Az Inception V3 konvolúciós rétegei:
0 input_1
1 convolution2d_1
2 batchnormalization_1
3 convolution2d_2
4 batchnormalization_2
5 convolution2d_3
6 batchnormalization_3
7 maxpooling2d_1
8 convolution2d_4
9 batchnormalization_4
10 convolution2d_5
11 batchnormalization_5
12 maxpooling2d_2
13 convolution2d_9
14 batchnormalization_9
15 convolution2d_7
16 convolution2d_10
17 batchnormalization_7
18 batchnormalization_10
19 averagepooling2d_1
20 convolution2d_6
21 convolution2d_8
22 convolution2d_11
23 convolution2d_12
24 batchnormalization_6
25 batchnormalization_8
26 batchnormalization_11
27 batchnormalization_12
28 mixed0
29 convolution2d_16
30 batchnormalization_16
31 convolution2d_14
32 convolution2d_17
33 batchnormalization_14
34 batchnormalization_17
35 averagepooling2d_2
36 convolution2d_13
37 convolution2d_15
38 convolution2d_18
39 convolution2d_19
40 batchnormalization_13
41 batchnormalization_15
42 batchnormalization_18
43 batchnormalization_19
44 mixed1
45 convolution2d_23
46 ba

In [12]:
# majd a hálónak csak az első 172 rétegét fagyasztjuk, a többit pedig engedjük tanulni
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

In [14]:
# ez után újra le kell fordítanunk a hálót, hogy most már az Inception V3 felsőbb rétegei tanuljanak
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# és ismét indítunk egy tanítást, ezúttal nem csak az előrecsatolt rétegek,
# hanem az Inception V3 felső rétegei is tovább tanulnak
model.fit(train_images, labels_onehot,  batch_size=16, nb_epoch=100, validation_split=0.2, callbacks=[history])
print("Tanítás vége.")

Train on 6400 samples, validate on 1600 samples
Epoch 1/100
6400/6400 [==============================] - 194s - loss: 1.7358 - acc: 0.5666 - val_loss: 8.6953 - val_acc: 0.1006
Epoch 2/100
6400/6400 [==============================] - 190s - loss: 1.6791 - acc: 0.5784 - val_loss: 8.8507 - val_acc: 0.1062
Epoch 3/100
6400/6400 [==============================] - 190s - loss: 1.6430 - acc: 0.5909 - val_loss: 8.8472 - val_acc: 0.0938
Epoch 4/100
6400/6400 [==============================] - 190s - loss: 1.5926 - acc: 0.5983 - val_loss: 8.9638 - val_acc: 0.1006
Epoch 5/100
6400/6400 [==============================] - 190s - loss: 1.5584 - acc: 0.6045 - val_loss: 8.9511 - val_acc: 0.1006
Epoch 6/100
6400/6400 [==============================] - 190s - loss: 1.5367 - acc: 0.6134 - val_loss: 9.0559 - val_acc: 0.0938
Epoch 7/100
6400/6400 [==============================] - 190s - loss: 1.4951 - acc: 0.6236 - val_loss: 8.9893 - val_acc: 0.0969
Epoch 8/100
6400/6400 [==============================] -

KeyboardInterrupt: 

In [81]:
predicted_values = model.predict(np.asarray(train_images)).shape

In [79]:
max_pred = np.zeros(predicted_values.shape, dtype=int)
festok = []
for i, pred in enumerate(predicted_values):
    d = np.argmax(pred)
    max_pred[i, d] = 1
    festok.append(d)
festok

[3, 7, 4, 13, 9, 8, 2, 1, 12, 5, 6, 12, 1, 3]

In [80]:
encoder.inverse_transform(festok)

array(['5b6a7be5ffc6a27b91bd3210ffa2e088',
       'cc47068929413a16aa707faefbdf4b70',
       '69904cf890070e9593a566394d5dece4',
       'f44205b1eb2de981de766e0688f8cbac',
       'da9ab2081b197129eeb91477d239be00',
       'd500fe452aef7a6f90de16197a9670bf',
       '485d901dc4df30b128bf01cb6e229767',
       '3f93b217bd0dbf874f973958f1eb6df4',
       'f14a3a6cc3112c9e92bc6c33c88eb264',
       '8e441c5899bf3d2f3b2c493e62fb92bf',
       'c56bcab4b317984013ebef5d3c4b5906',
       'f14a3a6cc3112c9e92bc6c33c88eb264',
       '3f93b217bd0dbf874f973958f1eb6df4',
       '5b6a7be5ffc6a27b91bd3210ffa2e088'], 
      dtype='<U32')

In [73]:
# vissza kéne alakítani valahogy...
ecovered_X = np.array([ohc.active_features_[col] for col in out.sorted_indices().indices])\
.reshape(n_samples, n_features) - ohc.feature_indices_[:-1]

NameError: name 'out' is not defined